In [ ]:
import os, sys 
import numpy as np 

IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)

from imatools.common import ioutils as iou
from imatools.common import vtktools as vtku 

def calc_cog(pts, el):
    return [np.mean(pts[ee], 0) for ee in el]

norm2 = lambda a : np.linalg.norm(a)
norm_vec = lambda a : a/norm2(a)

LABEL_STR = ['FULL', 'LA', 'LSPV', 'LIPV', 'RSPV', 'RIPV', 'LAA']
LABEL_LIST = [0, 1, 11, 13, 15, 17, 19]

In [ ]:
# Locations of hard drive based on platform
dirdic = {'macOS': '/Volumes/sandisk',
          'Linux': '/media/jsl19/sandisk',
          'Windows': 'D:/'}

tex_dic = {'macOS' : '/Users/jsolislemus/Documents/TEX', 
            'Linux' : '/home/jsl19/Documents/tex'}

base_dir = iou.fullfile(dirdic[iou.chooseplatform()], '01_atrialfibres/06_Reproducibility/05_UserProjects')
sims_dir = iou.fullfile(base_dir, '008_simulation_results')
out_dir = iou.fullfile(base_dir, '009_simulation_images', 'Fibre_Agreement')

tex_dir = iou.fullfile(tex_dic[iou.chooseplatform()], 'tex.cinbio.reproducibility/scripts/py')

N = ['M' + str(n) for n in np.linspace(1,100,num=100, dtype=int)]

In [ ]:
import pandas as pd 

df = pd.read_csv(iou.fullfile(base_dir, 'simulations_paths.csv'))
num_pairs = int(len(df)/2)

df

In [ ]:
import vtk 
def extract_from_dataframe(dframe, window, bdir) : 
    _dir = dframe.result_path[window]
    _user = dframe.user[window].tolist()
    _patient = dframe.patient[window].tolist()
    _mode = dframe['mode'][window].tolist()

    _dir = [px.replace('$AFIB_REPROD', bdir) for px in _dir]

    return _dir, _user, _patient, _mode

win_a = np.linspace(0,98,num=50,dtype=int)
win_b = np.linspace(1,99,num=50,dtype=int)

dir_a, user_a, patient_a, mode_a = extract_from_dataframe(df, window=win_a, bdir=base_dir)
dir_b, user_b, patient_b, mode_b = extract_from_dataframe(df, window=win_b, bdir=base_dir)

fibre_file = 'l' 
mname = 'Bilayer_' + fibre_file



In [ ]:

ix=1

u0 = user_a[ix] 
u1 = user_b[ix]
p0 = patient_a[ix] 
p1 = patient_b[ix]

iou.cout('{}: U0[{}] vs U1[{}]'.format(ix, u0, u1), typeMsg='PROCESSING')

patient = p0 if(mode_a[ix] == 'inter') else p0[0:-2]

mesh_a = vtku.readVtk(iou.fullfile(dir_a[ix], mname + '.vtk'))
mesh_b = vtku.readVtk(iou.fullfile(dir_b[ix], mname + '.vtk'))

fib_a = np.loadtxt(iou.fullfile(dir_a[ix], mname + '.lon'))
fib_b = np.loadtxt(iou.fullfile(dir_b[ix], mname + '.lon'))



# fib_comparison = vtku.compare_fibres(msh_a=mesh_a, msh_b=mesh_b, f_a=fib_a, f_b=fib_b)

In [ ]:
for ix in range(50) : 
    u0 = user_a[ix] 
    u1 = user_b[ix]
    p0 = patient_a[ix] 
    p1 = patient_b[ix]
    iou.cout('{}: U0[{}] vs U1[{}]'.format(ix, u0, u1), typeMsg='PROCESSING')

    patient = p0 if(mode_a[ix] == 'inter') else p0[0:-2]
    
    path_left = dir_a[ix]
    tot_left = iou.getTotal(iou.fullfile(path_left, mname+'.pts'))

    path_right = dir_b[ix]
    tot_right = iou.getTotal(iou.fullfile(path_right, mname + '.pts'))

    path0=path_left 
    path1=path_right 
    if tot_left < tot_right : 
        path0=path_right
        path1=path_left
    
    pts1, el1, region1 = iou.loadCarpMesh(mname, path1)
    pts1 /= 1000 
    cog1 = calc_cog(pts1, el1)

    f0 = np.loadtxt(iou.fullfile(path0, mname+'.lon'))
    f1 = np.loadtxt(iou.fullfile(path1, mname+'.lon'))

    pts0, el0, region0 = iou.loadCarpMesh(mname, path0)
    pts0 /= 1000 

    vtku.saveCarpAsVtk(pts0, el0, path0, mname)
    
    l0 = iou.fullfile(path0,mname+'.vtk')
    msh0 = vtku.readVtk(iou.fullfile(l0))

    cell_loc=vtk.vtkCellLocator()
    cell_loc.SetDataSet(msh0)
    cell_loc.BuildLocator()

    nel1 = len(el1)
    f0v1_dot = np.zeros(nel1)
    f0v1_dist = np.zeros(nel1)

    for jx in range(nel1):
        cellId = vtk.reference(0)
        c = [0.0, 0.0, 0.0]
        subId = vtk.reference(0)
        d = vtk.reference(0.0)

        cell_loc.FindClosestPoint(cog1[jx], c, cellId, subId, d)
        f0v1_dist[jx] = norm2(cog1[jx] - c)
        a=norm_vec(f0[cellId.get()])
        b=norm_vec(f1[jx])
        f0v1_dot[jx] = np.dot(a,b)

    f0v1_angles = np.arccos(f0v1_dot)
    f0v1_abs_dot = np.abs(f0v1_dot)
    f0v1_angle_abs_dot = np.arccos(f0v1_abs_dot)

    d={'region' : region1,
        'dot_product' : f0v1_dot,
        'angle' : f0v1_angles,
        'distance_to_point' : f0v1_dist,
        'abs_dot_product' : f0v1_abs_dot,
        'angle_from_absdot' : f0v1_angle_abs_dot}

    out_name = "fibres_agree_" + fibre_file + '_' + mode_a[ix] + "_" + patient + ".csv"
    out_df = pd.DataFrame(data=d)
    out_df.to_csv(iou.fullfile(out_dir, out_name))



In [ ]:
print(np.divide(f0.T*2,np.linalg.norm(f0*2, axis=1)).T)